#Databricks Delta Lake
En este notebook aprenderemos acerca de los Delta Lake y a como utilizarlo

### Datos de origen para este cuaderno
Los datos utilizados son una versión modificada de los datos públicos de [Lending Club] (https://www.kaggle.com/wendykan/lending-club-loan-data). Incluye todos los préstamos financiados de 2012 a 2017. Cada préstamo incluye la información del solicitante así como el estado actual del préstamo (actual, atrasado, pagado en su totalidad, etc.) y la información de pago más reciente.

## Setup

In [0]:
db = "deltadb"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[8]: DataFrame[key: string, value: string]

In [0]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *


def my_checkpoint_dir(): 
  return "/tmp/delta_demo/chkpt/%s" % str(random.randint(0, 10000))

# User-defined function to generate random state
@udf(returnType=StringType())
def random_state():
  return str(random.choice(["CA", "TX", "NY", "WA"]))


# Function to start a streaming query with a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream(table_format, table_name, schema_ok=False, type="batch"):
  
  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 500).load()
    .withColumn("loan_id", 10000 + col("value"))
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer"))
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000))
    .withColumn("addr_state", random_state())
    .withColumn("type", lit(type)))
    
  if schema_ok:
    stream_data = stream_data.select("loan_id", "funded_amnt", "paid_amnt", "addr_state", "type", "timestamp")
      
  query = (stream_data.writeStream
    .format(table_format)
    .option("checkpointLocation", my_checkpoint_dir())
    .trigger(processingTime = "5 seconds")
    .table(table_name))

  return query

In [0]:
# Function to stop all streaming queries 
def stop_all_streams():
    print("Stopping all streams")
    for s in spark.streams.active:
        try:
            s.stop()
        except:
            pass
    print("Stopped all streams")
    dbutils.fs.rm("/tmp/delta_demo/chkpt/", True)


def cleanup_paths_and_tables():
    dbutils.fs.rm("/tmp/delta_demo/", True)
    dbutils.fs.rm("file:/dbfs/tmp/delta_demo/loans_parquet/", True)
        
    for table in ["deltadb.loans_parquet", "deltadb.loans_delta", "deltadb.loans_delta2"]:
        spark.sql(f"DROP TABLE IF EXISTS {table}")
    
cleanup_paths_and_tables()

In [0]:
%sh mkdir -p /dbfs/tmp/delta_demo/loans_parquet/; wget -O /dbfs/tmp/delta_demo/loans_parquet/loans.parquet https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet

--2022-10-13 17:43:08--  https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.74.206, 104.17.70.206, 104.17.71.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.74.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’

     0K .......... .......... .......... .......... .......... 31% 9.57M 0s
    50K .......... .......... .......... .......... .......... 62% 5.62M 0s
   100K .......... .......... .......... .......... .......... 93% 6.37M 0s
   150K ..........                                            100% 68.5M=0.02s

2022-10-13 17:43:09 (7.26 MB/s) - ‘/dbfs/tmp/delta_demo/loans_parquet/loans.parquet’ saved [164631/164631]



# Empezando con <img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>

Una capa de almacenamiento de código abierto que trae transacciones ACID a Apache Spark ™ y cargas de trabajo de big data.

* **Transacciones ACID**: garantiza la integridad de los datos y la coherencia de lectura con datos concurrentes y complejos.
* **Sistema unificado para Streaming y Bacth**: La ingesta de datos en streaming, el reabastecimiento del histórico en batch y las consultas interactivas funcionan de inmediato para ambos sistemas de obtención de datos
* **Schema Enforcement and Evolution**: asegura la estructura de datos, y permite actualizarla bajo demanda facilmente
* **Recuperación de datos**: Consulta versiones anteriores de la tabla por tiempo o número de versión.
* **Elimina y actualiza**: admite la eliminación y la inserción en tablas con APIs.
* **Formato abierto**: almacenado como formato Parquet en el almacenamiento de blobs.
* **Historial de auditoría**: Historial de todas las operaciones que ocurrieron en la tabla.
* **Gestión de metadatos escalables**: Capaz de manejar millones de archivos, escalan las operaciones de metadatos con Spark.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Convertir a formato Delta Lake

Delta Lake es 100% compatible con Apache Spark & trade ;, lo que facilita comenzar si ya usa Spark para sus flujos de trabajo de big data.
Delta Lake cuenta con API para ** SQL **, ** Python ** y ** Scala **, para que pueda usarlo en cualquier idioma en el que se sienta más cómodo.

<img src="https://databricks.com/wp-content/uploads/2020/12/simplysaydelta.png" width=600/>

en ** Python **: lea sus datos en un Spark DataFrame, luego escríbalos en formato Delta Lake directamente, sin necesidad de una definición de esquema inicial.

In [0]:
parquet_path = "file:/dbfs/tmp/delta_demo/loans_parquet/"

df = (spark.read.format("parquet").load(parquet_path)
      .withColumn("type", lit("batch"))
      .withColumn("timestamp", current_timestamp()))

df.write.format("delta").mode("overwrite").saveAsTable("loans_delta")

** SQL: ** Use la declaración `CREATE TABLE` con SQL (no se necesita una definición de esquema inicial)

** SQL **: Use `CONVERT TO DELTA` para convertir archivos Parquet al formato Delta Lake en su lugar

### Ver los datos en la tabla de Delta Lake
** ¿Cuántos registros hay y cómo se ven los datos? **

In [0]:
spark.sql("select count(*) from loans_delta").show()
spark.sql("select * from loans_delta").show(3)

+--------+
|count(1)|
+--------+
|   14705|
+--------+

+-------+-----------+---------+----------+-----+--------------------+
|loan_id|funded_amnt|paid_amnt|addr_state| type|           timestamp|
+-------+-----------+---------+----------+-----+--------------------+
|      0|       1000|   182.22|        CA|batch|2022-10-13 17:43:...|
|      1|       1000|   361.19|        WA|batch|2022-10-13 17:43:...|
|      2|       1000|   176.26|        TX|batch|2022-10-13 17:43:...|
+-------+-----------+---------+----------+-----+--------------------+
only showing top 3 rows



## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unificar batch + streaming para procesamiento de datos con múltiples procesos de lectura y escritura

##### Escribe 2 flujos de datos diferentes en nuestra tabla de Delta Lake al mismo tiempo.

In [0]:
# Set up 2 streaming writes to our table
stream_query_A = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream A')
stream_query_B = generate_and_append_data_stream(table_format="delta", table_name="loans_delta", schema_ok=True, type='stream B')

##### Cree 2 lectores de transmisión continua de nuestro Delta Lake para ilustrar el progreso de la transmisión.

In [0]:
# Streaming read #1
display(spark.readStream.format("delta").table("loans_delta").groupBy("type").count().orderBy("type"))

type,count
batch,14705
stream A,2500


In [0]:
# Streaming read #2
display(spark.readStream.format("delta").table("loans_delta").groupBy("type", window("timestamp", "10 seconds")).count().orderBy("window"))

type,window,count
batch,"List(2022-10-13T17:43:00.000+0000, 2022-10-13T17:43:10.000+0000)",14705
stream A,"List(2022-10-13T17:43:10.000+0000, 2022-10-13T17:43:20.000+0000)",304
stream A,"List(2022-10-13T17:43:20.000+0000, 2022-10-13T17:43:30.000+0000)",5000
stream B,"List(2022-10-13T17:43:20.000+0000, 2022-10-13T17:43:30.000+0000)",4699
stream B,"List(2022-10-13T17:43:30.000+0000, 2022-10-13T17:43:40.000+0000)",4801
stream A,"List(2022-10-13T17:43:30.000+0000, 2022-10-13T17:43:40.000+0000)",2696


##### Agrega una consulta por lotes para comprobar el funcionamiento

In [0]:
%sql
SELECT addr_state, COUNT(*)
FROM loans_delta
GROUP BY addr_state

addr_state,count(1)
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


In [0]:
dbutils.notebook.exit("stop")

stop

In [0]:
stop_all_streams()

Stopping all streams
Stopped all streams


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Transacciones ACID

Ver el registro de transacciones de Delta Lake

In [0]:
%sql DESCRIBE HISTORY loans_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
8,2022-10-13T17:45:37.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> e5041f41-0a1e-41c2-bfa2-09142c844e04, epochId -> 3)",null,List(3160246529845061),1013-173158-cutpmxav,6,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 39000, numOutputBytes -> 872854, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
7,2022-10-13T17:45:36.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> dc2593d4-8ce8-44e1-9cad-4c17f6b2ec46, epochId -> 3)",null,List(3160246529845061),1013-173158-cutpmxav,6,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 38000, numOutputBytes -> 851285, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-10-13T17:44:53.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> e5041f41-0a1e-41c2-bfa2-09142c844e04, epochId -> 2)",null,List(3160246529845061),1013-173158-cutpmxav,4,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5000, numOutputBytes -> 123250, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-10-13T17:43:41.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> dc2593d4-8ce8-44e1-9cad-4c17f6b2ec46, epochId -> 2)",null,List(3160246529845061),1013-173158-cutpmxav,4,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 5500, numOutputBytes -> 136828, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-10-13T17:43:36.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> e5041f41-0a1e-41c2-bfa2-09142c844e04, epochId -> 1)",null,List(3160246529845061),1013-173158-cutpmxav,2,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 4500, numOutputBytes -> 112756, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-10-13T17:43:32.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> dc2593d4-8ce8-44e1-9cad-4c17f6b2ec46, epochId -> 1)",null,List(3160246529845061),1013-173158-cutpmxav,2,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 2500, numOutputBytes -> 63342, numAddedFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-10-13T17:43:25.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> e5041f41-0a1e-41c2-bfa2-09142c844e04, epochId -> 0)",null,List(3160246529845061),1013-173158-cutpmxav,1,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 0, numOutputBytes -> 0, numAddedFiles -> 0)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-10-13T17:43:22.000+0000,3782011831695806,juanrry@hotmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> dc2593d4-8ce8-44e1-9cad-4c17f6b2ec46, epochId -> 0)",null,List(3160246529845061),1013-173158-cutpmxav,0,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 0, numOutputBytes -> 0, numAddedFiles -> 0)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-10-13T17:43:12.000+0000,3782011831695806,juanrry@hotmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {""delta.autoOptimize.optimizeWrite"":""true""})",null,List(3160246529845061),1013-173158-cutpmxav,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 14705, numOutputBytes -> 165506)",null,Databricks-Runtime/10.4.x-scala2.12


<img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=1012/>

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Utilice Schema Enforcement para proteger la calidad de los datos

Para mostrarle cómo funciona la aplicación del esquema, creemos una nueva tabla que tenga una columna adicional, `credit_score`, que no coincide con nuestro esquema de tabla de Delta Lake existente.

###### Escribe DataFrame con una columna adicional, `credit_score`, en la tabla de Delta Lake

In [0]:
# Generate `new_data` with additional column
new_column = [StructField("credit_score", IntegerType(), True)]

new_schema = StructType(spark.table("loans_delta").schema.fields + new_column)
data = [(99997, 10000, 1338.55, "CA", "batch", datetime.now(), 649),
        (99998, 20000, 1442.55, "NY", "batch", datetime.now(), 702)]

new_data = spark.createDataFrame(data, new_schema)
new_data.printSchema()

In [0]:
# Uncommenting this cell will lead to an error because the schemas don't match.
# Attempt to write data with new column to Delta Lake table
new_data.write.format("delta").mode("append").saveAsTable("loans_delta")

**La aplicación del esquema ayuda a mantener nuestras tablas limpias y ordenadas para que podamos confiar en los datos que hemos almacenado en Delta Lake.** Las escrituras anteriores se bloquearon porque el esquema de los nuevos datos no coincidía con el esquema de la tabla

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Utilice Schema Evolution para agregar nuevas columnas al esquema

Si *queremos* actualizar nuestra tabla de Delta Lake para que coincida con el esquema de esta fuente de datos, podemos hacerlo mediante la evolución del esquema. Simplemente agregue lo siguiente al comando de escritura de Spark: `.option (" mergeSchema "," true ")`

In [0]:
new_data.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("loans_delta")

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (99997, 99998)

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake Time Travel, recuperación de versiones

Las capacidades de Delta Lake simplifican la creación de canalizaciones de datos para casos de uso que incluyen:

* Auditoría de cambios de datos
* Reproducción de experimentos e informes
* Retrocesos

A medida que escribe en una tabla o directorio Delta, cada operación se versiona automáticamente.

<img src="https://github.com/risan4841/img/blob/master/transactionallogs.png?raw=true" width=250/>

Puedes consultar instantáneas de sus tablas con:
1. **Version number**, 
2. **Timestamp.**

utilizando sintaxis de Python, Scala y / o SQL; para estos ejemplos usaremos la sintaxis SQL.

#### Revisar el historial de tablas de Delta Lake para auditoría y gobernanza
Todas las transacciones para esta tabla se almacenan dentro de esta tabla, incluido el conjunto inicial de inserciones, actualización, eliminación, fusión e inserciones con modificación de esquema

In [0]:
%sql
DESCRIBE HISTORY loans_delta

###### Usa el viaje en el tiempo para seleccionar y ver la versión original de nuestra tabla (Versión 0).
Como puede ver, esta versión contiene los 14.705 registros originales.

In [0]:
spark.sql("SELECT * FROM loans_delta VERSION AS OF 0").show(3)
spark.sql("SELECT COUNT(*) FROM loans_delta VERSION AS OF 0").show()

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

###### Revierte una tabla a una versión específica usando `RESTORE`

In [0]:
%sql RESTORE loans_delta VERSION AS OF 0

In [0]:
%sql SELECT COUNT(*) FROM loans_delta

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Soporte completo de DML: `DELETE`,` UPDATE`, `MERGE INTO`

Delta Lake trae transacciones ACID y soporte DML completo a los lagos de datos.

> Parquet **no** admite estos comandos, son exclusivos de Delta Lake.

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `DELETE`: Maneje las solicitudes de GDPR o CCPA en su Data Lake

Imagínese que estamos respondiendo a una solicitud de eliminación de datos de GDPR. El usuario con ID de préstamo # 4420 quiere que eliminemos sus datos. Así de fácil es.

In [0]:
%sql
SELECT * FROM loans_delta WHERE loan_id=4420

**Elimine los datos del usuario individual con un solo comando `DELETE` usando Delta Lake.**

Nota: El comando `DELETE` no es compatible con Parquet.

In [0]:
%sql
DELETE FROM loans_delta WHERE loan_id=4420;
-- Confirm the user's data was deleted
SELECT * FROM loans_delta WHERE loan_id=4420

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Utilice el viaje en el tiempo e `INSERT INTO` para volver a agregar al usuario a nuestra tabla

In [0]:
%sql
INSERT INTO loans_delta
SELECT * FROM loans_delta VERSION AS OF 0
WHERE loan_id=4420

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id=4420

### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) `UPDATE`: Modifica los registros existentes en una tabla en un comando

In [0]:
%sql UPDATE loans_delta SET funded_amnt = 22000 WHERE loan_id = 4420

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id = 4420

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Admite flujos de trabajo de captura de datos modificados y otros casos de uso de ingesta a través de `MERGE INTO`

Con una canalización de datos heredada, para insertar o actualizar una tabla, debe:
1. Identifique las nuevas filas que se insertarán
2. Identifique las filas que serán reemplazadas (es decir, actualizadas)
3. Identifique todas las filas que no se ven afectadas por la inserción o actualización.
4. Cree una nueva tabla basada en las tres declaraciones de inserción
5. Elimina la tabla original (y todos esos archivos asociados).
6. "Cambiar el nombre" de la tabla temporal al nombre de la tabla original
7. Suelta la tabla temporal

<img src="https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif" alt='Merge process' width=600/>


#### INSERTAR o ACTUALIZAR con Delta Lake

Proceso de 2 pasos:
1. Identificar filas para insertar o actualizar
2. Utilice "MERGE"

In [0]:
# Create merge table with 1 row update, 1 insertion
data = [(4420, 22000, 21500.00, "NY", "update", datetime.now()),  # record to update
        (99999, 10000, 1338.55, "CA", "insert", datetime.now())]  # record to insert
schema = spark.table("loans_delta").schema
spark.createDataFrame(data, schema).createOrReplaceTempView("merge_table")
spark.sql("SELECT * FROM merge_table").show()

In [0]:
%sql
MERGE INTO loans_delta AS l
USING merge_table AS m
ON l.loan_id = m.loan_id
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

In [0]:
%sql SELECT * FROM loans_delta WHERE loan_id IN (4420, 99999)

## ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Optimizaciones de rendimiento y compactación de archivos = consultas más rápidas

In [0]:
%sql
-- Vacuum deletes all files no longer needed by the current version of the table.
VACUUM loans_delta

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Tabla de caché en la memoria (solo Databricks Delta Lake)

In [0]:
%sql CACHE SELECT * FROM loans_delta

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Optimización de orden Z (solo Databricks Delta Lake)

In [0]:
%sql OPTIMIZE loans_delta ZORDER BY addr_state

In [0]:
cleanup_paths_and_tables()

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>